In [1]:
from pynq import Overlay
from pynq import allocate
import numpy as np
from time import perf_counter
import time
import matplotlib.pyplot as plt
import pynq
from pynq import get_rails, DataRecorder
from datetime import datetime
import argparse
import threading

In [13]:
def matmul(
    A_local,
    B_local,
    output_local,
    PL):
    
    start=0x10
    PL.write(start, A_local.device_address)
    PL.write(start+12, B_local.device_address)
    PL.write(start+12*2, output_local.device_address)
    
    ctrl = PL.read(0x00)
    PL.write(0x00, (ctrl & 0x80) | 0x01)
    while (PL.read(0x00) & 0x02) == 0:
        pass
    
def matmul2(
    A_local,
    B_local,
    output_local,
    PL):
    
    start=0x10
    PL.register_map.A_outer_1.A_outer = A_local.device_address
    PL.register_map.B_outer_1.B_outer = B_local.device_address
    PL.register_map.output_outer_1.output_outer = output_local.device_address
    
    ctrl = PL.read(0x00)
    PL.write(0x00, (ctrl & 0x80) | 0x01)
    while (PL.read(0x00) & 0x02) == 0:
        pass

In [17]:
overlay = Overlay('design_1.bit')
#v2.7
ip = overlay.matmul_0.s_axi_control

In [4]:
ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  A_outer_1 = Register(A_outer=write-only),
  A_outer_2 = Register(A_outer=write-only),
  B_outer_1 = Register(B_outer=write-only),
  B_outer_2 = Register(B_outer=write-only),
  output_outer_1 = Register(output_outer=write-only),
  output_outer_2 = Register(output_outer=write-only)
}

In [5]:
SIZE = 2
A = allocate(shape=(SIZE, SIZE), dtype=np.float16)
B = allocate(shape=(SIZE, SIZE), dtype=np.float16)
output = allocate(shape=(SIZE, SIZE), dtype=np.float16)

In [11]:
#検証用
np.random.seed(seed=32)
A_valid = np.random.rand(SIZE, SIZE).astype(np.float16)
B_valid = np.random.rand(SIZE, SIZE).astype(np.float16)
output_valid = np.multiply(A_valid, B_valid)

In [7]:
for i in range(SIZE):
    for j in range(SIZE):
        A[i][j] = A_valid[i][j]
        B[i][j] = B_valid[i][j]

In [14]:
start_time = perf_counter()
matmul2(A, B, output, ip)
end_time = perf_counter()
print('time=', end_time-start_time)

time= 0.0007144400001379836


In [18]:
output

PynqBuffer([[0., 0.],
            [0., 0.]], dtype=float16)

In [12]:
output_valid

array([[0.633  , 0.3044 ],
       [0.05612, 0.887  ]], dtype=float16)